In [1]:
import datasets
from datasets import list_datasets, load_dataset, list_metrics, load_metric
datasets_list = list_datasets()
import torch
import re
#import bleach
#from bs4 import BeautifulSoup

In [ ]:
dataset  = datasets.load_dataset('hyperpartisan_news_detection', 'bypublisher')
dataset


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

In [1]:
dataset['train']

NameError: name 'dataset' is not defined

In [ ]:
def clean_text(text, label):
    """ Clean the input text and remove special characters """
#     soup = BeautifulSoup(text)
    text = bleach.clean(text,strip=True)
    text = text.replace('<p>', '')
    text = text.replace('</p>', '')
    text = text.replace('\n', '')
    text = text.replace('&amp;#160;', '')
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    if str(label) == 'True':
        new_label = 1
    else:
        new_label = 0
    return text, new_label


def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    text_, target_ = clean_text(example['text'], example['hyperpartisan'])
    encodings = tokenizer.encode_plus(text_, pad_to_max_length=True, max_length=2048,
                                           add_special_tokens=True,
                                            return_token_type_ids=False,
                                            return_attention_mask=True,
                                            padding='max_length', truncation=True,
                                           )
    targets = torch.tensor(target_,dtype=torch.long)
    

    encodings.update({'targets': targets,
                      'attention_mask': encodings['attention_mask']})
    return encodings

In [ ]:
from transformers import LongformerTokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')